https://python.langchain.com/docs/get_started/quickstart/

In [ ]:
#!pip install langchain
#!pip install langchain-openai

In [11]:
from dotenv import load_dotenv
load_dotenv()
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="..."
# export OPENAI_API_KEY="..."

True

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI()
output_parser = StrOutputParser()

# Prompt engineering 

In [8]:
llm.invoke("I have been experiencing a headache and dizziness for the last few days.")

AIMessage(content="I'm sorry to hear that you're not feeling well. It's important to take care of yourself and address your symptoms. Have you been able to see a doctor or healthcare provider about your headache and dizziness? It's possible that they could be related to a number of different factors, such as dehydration, stress, or an underlying medical condition. I recommend seeking medical advice to determine the cause of your symptoms and receive appropriate treatment. In the meantime, make sure to stay hydrated, get plenty of rest, and avoid any activities that worsen your symptoms. Take care!", response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 22, 'total_tokens': 137}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-fa0b48ce-b6d0-40b6-98da-8c976672103a-0')

In [18]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "I want you to act as a virtual doctor. I will describe my symptoms and \
    you will provide a diagnosis and treatment plan.\
    You should only reply with your diagnosis and treatment plan, and nothing else."),
    ("user", "{input}")
])
chain = prompt | llm 
chain.invoke({"input": "I have been experiencing a headache and dizziness for the last few days."})

AIMessage(content='Diagnosis: Possible migraine headache\nTreatment plan: Rest in a quiet, dark room, stay hydrated, over-the-counter pain relievers like acetaminophen or ibuprofen, consider seeing a doctor for further evaluation and prescription medications if symptoms persist.', response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 68, 'total_tokens': 119}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-50d303ce-92af-4781-a2d6-0252feb580ea-0')

In [19]:
chain = prompt | llm | output_parser
print(chain.invoke({"input": "I have been experiencing a headache and dizziness for the last few days."}))

Diagnosis: Possible migraine headache with associated dizziness

Treatment Plan:
1. Rest in a quiet, dark room
2. Stay hydrated and maintain a regular sleep schedule
3. Over-the-counter pain relief medication like acetaminophen or ibuprofen
4. Avoid triggers such as certain foods, stress, or lack of sleep
5. If symptoms persist or worsen, consult a healthcare provider


In [29]:
print(chain.invoke({"input": "I have been experiencing mild fever and exhaustion for the last few days."}))

Diagnosis: Possible viral infection or flu

Treatment plan: Get plenty of rest, stay hydrated, take over-the-counter pain relievers like acetaminophen or ibuprofen for fever and body aches. If symptoms persist or worsen, consult a healthcare provider.


# Retrieval Chain

In [14]:
#!pip install beautifulsoup4
#!pip install faiss-cpu

In [20]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.webmd.com/cold-and-flu/flu-cold-symptoms")

docs = loader.load()

In [21]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [23]:
#docs

In [24]:
# local vectorstore, FAISS https://python.langchain.com/docs/integrations/vectorstores/faiss/
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [26]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""I want you to act as a virtual doctor. 
I will describe my symptoms and you will provide a diagnosis and treatment plan based on the provided context:

<context>
{context}
</context>

Symptoms: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [27]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "I have been experiencing mild fever and exhaustion for the last few days."})
print(response["answer"])


Diagnosis: Based on the symptoms of mild fever and exhaustion, it is possible that you may have contracted either a cold or flu virus.

Treatment Plan: 
1. Rest: Make sure to get plenty of rest to allow your body to recover.
2. Stay Hydrated: Drink plenty of fluids to help with fever and exhaustion.
3. Over-the-counter Medications: Consider taking over-the-counter pain relievers and fever reducers to help alleviate symptoms.
4. Monitor Symptoms: Keep an eye on your symptoms and if they worsen or persist for more than a week, consider contacting a doctor for further evaluation and possible treatment options.

Remember to follow good hygiene practices, such as frequent handwashing, to prevent spreading the virus to others.
